In [1]:
!pip install pymed

In [2]:
!pip install entrezpy

     |████████████████████████████████| 97 kB 716 kB/s eta 0:00:01


In [14]:
!pip install biopython

     |████████████████████████████████| 2.2 MB 146 kB/s eta 0:00:01


In [1]:
import Bio

In [2]:
Bio.__version__

'1.79.dev0'

In [21]:
from Bio import Entrez
import xml.etree.ElementTree as ET 
from bs4 import BeautifulSoup 

In [2]:
Entrez.email = 'jpzhangv@gmail.com'

In [3]:
search_query = '("neurofibromatoses"[MeSH Terms] OR "neurofibromatoses"[All Fields] OR "neurofibromatosis"[All Fields]) AND NF2[All Fields] AND "open access"[filter]'

In [4]:
search_results = Entrez.read(Entrez.esearch(db="pmc", term=search_query, retmax=10, usehistory="y"))

In [5]:
search_results["Count"]

'1200'

In [14]:
handle = Entrez.efetch(db="pmc", rettype="full", retmode="xml", retstart=0, retmax=int(search_results["Count"]), webenv=search_results["WebEnv"], query_key=search_results["QueryKey"])

In [54]:
#record = Entrez.read(handle, validate=False)

In [16]:
text = handle.read()
type(text)

bytes

In [19]:
len(text)

186670066

In [24]:
text[:10000]

b'<?xml version="1.0" ?>\n<!DOCTYPE pmc-articleset PUBLIC "-//NLM//DTD ARTICLE SET 2.0//EN" "https://dtd.nlm.nih.gov/ncbi/pmc/articleset/nlm-articleset-2.0.dtd">\n<pmc-articleset><article xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:mml="http://www.w3.org/1998/Math/MathML" article-type="research-article">\n  <?properties open_access?>\n  <front>\n    <journal-meta>\n      <journal-id journal-id-type="nlm-ta">JCI Insight</journal-id>\n      <journal-id journal-id-type="iso-abbrev">JCI Insight</journal-id>\n      <journal-id journal-id-type="publisher-id">JCI Insight</journal-id>\n      <journal-title-group>\n        <journal-title>JCI Insight</journal-title>\n      </journal-title-group>\n      <issn pub-type="epub">2379-3708</issn>\n      <publisher>\n        <publisher-name>American Society for Clinical Investigation</publisher-name>\n      </publisher>\n    </journal-meta>\n    <article-meta>\n      <article-id pub-id-type="pmid">32960816</article-id>\n      <article-id pub-id-ty

In [17]:
text[-2000:]

b'tes, one was derived from a neurofibromatosis patient and the other from an A-T heterozygote. When SV40 DNA was employed as the transforming agent for the latter, the transformation rate was no longer raised.</p>\n        <sec sec-type="scanned-figures">\n          <title>Images</title>\n          <fig id="F1">\n            <label>Fig. 1</label>\n            <graphic xlink:href="brjcancer00296-0052-a" xlink:role="586"/>\n          </fig>\n          <fig id="F2">\n            <label>Fig. 2</label>\n            <graphic xlink:href="brjcancer00296-0053-a" xlink:role="587"/>\n          </fig>\n        </sec>\n      </abstract>\n    </article-meta>\n  </front>\n  <body>\n    <supplementary-material content-type="scanned-pages">\n      <graphic xlink:href="brjcancer00296-0049.tif" xlink:title="scanned-page" xlink:role="583" mimetype="image" mime-subtype="tiff"/>\n      <graphic xlink:href="brjcancer00296-0050.tif" xlink:title="scanned-page" xlink:role="584" mimetype="image" mime-subtype="t

In [22]:
text.count(b'article-id pub-id-type="pmc"')

1200

In [28]:
text.count(b'<abstract>')

921

In [63]:
text.count(b'<article-title>')

75296

In [26]:
text.count(b'article-title')

150637

In [22]:
pmc_paper_data = BeautifulSoup(text, "xml") 

In [30]:
set([tag.name for tag in pmc_paper_data.find_all()])

{'abstract',
 'ack',
 'addr-line',
 'address',
 'aff',
 'alt-text',
 'alt-title',
 'alternatives',
 'annotation',
 'anonymous',
 'app',
 'app-group',
 'array',
 'article',
 'article-categories',
 'article-id',
 'article-meta',
 'article-title',
 'article-version',
 'attrib',
 'author-comment',
 'author-notes',
 'award-group',
 'award-id',
 'back',
 'bio',
 'body',
 'bold',
 'boxed-text',
 'break',
 'caption',
 'chapter-title',
 'citation',
 'city',
 'col',
 'colgroup',
 'collab',
 'comment',
 'compound-subject',
 'compound-subject-part',
 'conf-date',
 'conf-loc',
 'conf-name',
 'conference',
 'contrib',
 'contrib-group',
 'contrib-id',
 'copyright-holder',
 'copyright-statement',
 'copyright-year',
 'corresp',
 'country',
 'counts',
 'custom-meta',
 'custom-meta-group',
 'date',
 'date-in-citation',
 'day',
 'def',
 'def-item',
 'def-list',
 'degrees',
 'disp-formula',
 'disp-quote',
 'edition',
 'element-citation',
 'elocation-id',
 'email',
 'equation-count',
 'etal',
 'ext-link',
 

In [70]:
len(pmc_paper_data.find_all('article'))

1200

In [71]:
len(pmc_paper_data.find_all('abstract'))

1282

In [59]:
#pmc_paper_data.find_all('article-id', attrs={'pub-id-type' : 'pmc'})
#./pmc-articleset/article/ [@pub-id-type="pmc"]

In [60]:
all_pmc_ids = [tag.text for tag in pmc_paper_data.find_all('article-id', attrs={'pub-id-type' : 'pmc'})]

In [75]:
all_article = pmc_paper_data.find_all('article')

In [132]:
for article in all_article[:10]:
    pmd = article.find("article-id", attrs={'pub-id-type' : 'pmc'}).getText()
    title = article.find("article-title").getText()
    author_ls = set([s.getText().strip().replace("\n", " ") for ss in article.find_all("contrib", attrs = {'contrib-type': "author"}) for s in ss.find_all("name")])
    authors = ', '.join(author_ls)
    affliations = 
    abstract_node = article.find("abstract")
    if abstract_node:
        abstract = abstract_node.find("p").getText().strip()
    else:
        abstract = ''
        
    print(authors)
    print("\n")

Chen Zhiguo, Wang Yong, Mo Juan, Hawley Eric, Shipman Tracey, Le Lu Q., Brosseau Jean-Philippe, Carroll Thomas J., Li Stephen, He Yongzheng, Clapp D. Wade


Napolitano Filomena, Melone Mariarosa Anna Beatrice, Riccardi Claudia, Perrone Lorena, Sampaolo Simone


Temel Yasin, Hovinga Koos E.


Kirby Amelia, Guzman Miguel, Pinz Hailey, Navalkele Pournima, Kemp Joanna


Božanić Urbančič Nina, Battelino Saba, Tesovnik Tine, Trebušak Podkrajšek Katarina


Paats Marthe S., Veerman G.D. Marijn, van den Bent Martin J.


Gugel Isabel, Tatagiba Marcos, Zipfel Julian, Kluwe Lan, Schuhmann Martin Ulrich, Hartjen Philip, Mautner Victor-Felix


Grimm Florian, Gugel Isabel, Teuber Christian, Tatagiba Marcos, Zipfel Julian, Kluwe Lan, Schuhmann Martin Ulrich, Mautner Victor-Felix


Gray Robert, Hamilton Stanley R, Takebe Naoko, Sims David J, Li Shuli, Mitchell Edith P, Harris Lyndsay N, Rubinstein Larry V, Comis Robert, Catalano Robert, Flaherty Keith T, Zwiebel James A, Kibbe Warren, Schwartz Gary K, 

In [155]:
[t.text for t in all_article[3].find_all("aff")]

['n1\nDepartment of Pediatrics, Cardinal Glennon Children’s Hospital, St. Louis University, St. Louis, Missouri',
 'n2\nDepartment of Pathology, Cardinal Glennon Children’s Hospital, St. University, St. Louis University, St. Louis, Missouri',
 'n3\nDepartment of Pediatrics, Wake Forest School of Medicine, Winston-Salem, North Carolina',
 'n4\nDepartment of Genetics, Cardinal Glennon Children’s Hospital, St. Louis University, St. Louis, Missouri',
 'n5\nDepartment of Neurosurgery, Saint Louis University Hospital, St. Louis University, St. Louis, Missouri']

In [149]:
[t.text for tt in all_article[112].find_all("aff") for t in tt.find_all('institution')]

['Paediatrics and Adolescent Medicine, Medical Faculty, University of Augsburg',
 'Department of Clinical Psychology, Helmut Schmidt University',
 'Institute of Neuropathology, DGNN Brain Tumor Reference Center, University Bonn',
 'Institute of Diagnostic and Interventional Neuroradiology, University Hospital Wuerzburg',
 'Section of Pediatric Neurosurgery, University Hospital Wuerzburg',
 'Department of Radio-Oncology, University Leipzig',
 'Department of Particle Therapy, University Hospital Essen, West German Proton Therapy Centre Essen (WPE), West German Cancer Center (WTZ), German Cancer Consortium (DKTK)',
 'Pediatric Neurosurgery, Charité Universitaetsmedizin Berlin',
 'Hopp Children’s Cancer Center Heidelberg (KiTZ), German Cancer Research Center (DKFZ) and Heidelberg University Hospital',
 'University Medical Center Hamburg-Eppendorf',
 'Department of Pediatric Oncology/Hematology, Charité-Universitaetsmedizin Berlin, Corporate member of Freie Universitaet Berlin, Humboldt-Uni

In [130]:

set([s.getText().strip().replace("\n", " ") for ss in all_article[1].find_all("contrib", attrs = {'contrib-type': "author"}) for s in ss.find_all("name")])


{'Melone Mariarosa Anna Beatrice',
 'Napolitano Filomena',
 'Perrone Lorena',
 'Riccardi Claudia',
 'Sampaolo Simone'}

In [61]:
len(all_pmc_ids)

1200

In [55]:
all_titles = [tag.text for tag in pmc_paper_data.find_all('article-title')]

In [57]:
type(pmc_paper_data)

bs4.BeautifulSoup

In [56]:
len(all_titles)

75321

In [ ]:
for movie in pmc_paper_data.findall("article-id"):
    print(movie.attrib)